# Descrição do experimento:
 
Esse experimento terá a seguinte estrutura:

Modelo LLM: LLama3.1

Modelo Embedding: nomic-embed-text	

Texto chunkenizado: não

EIOS Resumido: não

Notícia Resumida: não

Método de escolha: sem escolha (passando todo o contexto)

Tamanho de entrada do Llama3.1: 128k

Tamanho médio do prompt usando todos os textos do EIOS Test: +900k

Média de palavras por linha no dataset de teste: 506.07

# Experimento

In [1]:
#Lendo datasets e configurando numpy para float64 

from tqdm import tqdm
import pandas as pd
import numpy as np

np.float_ = np.float64

data = pd.read_csv('EIOS_train.csv')
train = pd.DataFrame(data)
data = pd.read_csv('EIOS_test.csv')
test = pd.DataFrame(data)
print(len(test))
# print(train.label.value_counts())

1890


In [8]:
# pegando a média de palavras por linha

total = len(test)

soma = ''

for text in test.text:
    soma += text

print(f"{len(soma)/total:.2f}")

506.07


In [3]:
# Definindo o modelo de LLM

from langchain_community.llms import Ollama
llm = Ollama(model = "llama3.1")

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [4]:
# Definindo o modelo gerador de embeddings

from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [10]:
# Carregando os documentos, e gerando os embeddings a partir dos documentos

from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import DataFrameLoader

# Carregando os documentos de treino
loader = DataFrameLoader(test, page_content_column="text") 
docs = loader.load()

# Gerando os embeddings
# vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)


In [15]:
import ollama
import ast 

# Define the function to call the Ollama Llama3.1 model
def ollama_llm(question, context):
    prompt_specs = """
        Classifique se as questões são relevantes ou irrelevantes para o contexto da área de saúde.
        Pense da perspectiva de um profissional de saúde como um médico, mas também como um epidemiologista, enfermeiro, ou outro profissional de saúde.

        Você deve classificar cada questão em uma das 2 categorias:
        1. Relevante
        2. Irrelevante

        Para cada questão, forneça uma justificativa detalhada para a escolha feita.
        Certifique-se de que a justificativa seja clara e esteja relacionada ao contexto da área da saúde.

        Faça:
        1. Seja específico e claro.
        2. Compreenda o contexto da questão relacionada à saúde antes de categorizá-la.
        3. Aja como um profissional de saúde.

        NÃO faça:
        1. Não adivinhe ou invente informações.
        2. Não crie novas categorias de justificativa; utilize apenas as fornecidas acima.

        Dica: Classifique como irrelevante quaisquer questões que não estejam diretamente relacionadas à saúde ou cuidados médicos.

        O formato de saída deve ser o seguinte:
        {
            "classe": "",
            "justificativa": ""
        }

        A classe deve ser uma string das categorias acima: 'Relevante' ou 'Irrelevante'.
        A justificativa deve ser um texto explicando o motivo da escolha da categoria.

        Exemplos:
        1. Questão: 'Tenho dores pulmonares afetadas pelo cigarro.'
            Saída: { 'classe': 'Relevante', 'justificativa': 'O texto trata de um problema respiratório diretamente relacionado à área da saúde.' }

        2. Questão: 'Como posso aumentar minha produtividade no trabalho?'
            Saída: { 'classe': 'Irrelevante', 'justificativa': 'A questão não está relacionada diretamente ao contexto da área da saúde.' }

        3. Questão: 'Tenho dores nas costas após longas horas sentado.'
            Saída: { 'classe': 'Relevante', 'justificativa': 'A questão aborda um problema de saúde ocupacional que afeta a coluna vertebral.' }

        4. Questão: 'Quais são os melhores livros de autoajuda?'
            Saída: { 'classe': 'Irrelevante', 'justificativa': 'A questão não se relaciona diretamente com o contexto de saúde ou cuidados médicos.' }
        
        Não se esqueça das orientações e forneça a resposta no formato de saída indicado.

        A saída deve ser estritamente no formato indicado, com aspas simples e chaves.

        A classificação deve ser exclusivamente entre 'Relevante' e 'Irrelevante'.
    """
    
    formatted_prompt = f"{prompt_specs}\nQuestão: {question}\n\nContexto: {context}"
    
    # with open('formatted_prompt.txt', 'w') as file:       
    #     file.write(formatted_prompt)
    # file.close()

    response = ollama.chat(model='llama3.1', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

def rag_chain(question):
    all_docs_content = "\n\n".join([doc.page_content for doc in docs])
    return ollama_llm(question, all_docs_content)

In [7]:
def redimensionar_df(df, n):
    data_test = []

    for _, row in df.iterrows():
        data_test.append(row['text'])
        if len(data_test) == n:
            break

    df_data_test = pd.DataFrame(data_test, columns=['text'])
    return df_data_test

df_test = redimensionar_df(train, 200)

In [8]:
def run_test(df):
    correct = 0
    for text in tqdm(df['text']):
        # print(text)
        response = rag_chain(text)
        response = ast.literal_eval(response)
        choice = response['classe'].lower()
        if choice == 'relevante':
            correct += 1
    return correct

In [16]:
correct_test = run_test(df_test)

  0%|          | 0/200 [00:35<?, ?it/s]


SyntaxError: invalid syntax (<unknown>, line 1)

In [ ]:
print(f"Correct: {correct_test}/{len(df_test)}")
print(f"Accuracy: {correct_test/len(df_test)}")